In [ ]:
import sqlite3
from ydata_profiling import ProfileReport
import polars as pl
import pandas as pd
import numpy as np
import re
import os

In [ ]:
save_location = ""

In [ ]:
def codes_to_names(tab_to_process, codes_tab, shortname):
    errors = []
    codes_group = codes_tab.filter(pl.col("form") == shortname)
    try:
        codes_group = codes_group.with_columns(pl.col("value").cast(pl.Int64))
    except Exception:
        pass
    codes_group = codes_group.filter(pl.col("value").is_not_null())
    if codes_group.height == 0:
        print("NO VARS TO RENAME IN:", shortname)
        return tab_to_process, errors
    for varname, vartab in codes_group.group_by("variable"):
        if varname[0] == "fenotipo_resist_colo":
            varname = ["feno_resist_colo"]
        column_dict = vartab.select(["value", "name"]).to_dict(as_series=False)
        codes2names_dict = {
            item[0]: item[1] for item in zip(column_dict["value"], column_dict["name"])
        }
        column = varname[0]
        print(f"RENAMING {column} for {codes2names_dict}:")
        try:
            tab_to_process = tab_to_process.with_columns(pl.col(column)).cast(pl.String)
            # print(tab_to_process.head())
            tab_to_process = tab_to_process.with_columns(
                pl.Series(tab_to_process[column]).replace(codes2names_dict)
            )
            # print("AFTER RENAMING", tab_to_process.head())
        except Exception as e:
            errors.append(
                f"Could not rename {column}. Columns found: {tab_to_process.columns}. Error: {e}"
            )
            continue
    return tab_to_process, errors

db_path = os.path.join(save_location, "db_mepram_sepsis_v2.sqlite3")
con = sqlite3.connect(db_path)
tablenames = pl.read_database(
    "SELECT name FROM sqlite_master WHERE type = 'table'", con
)
tablenames = sorted([x for x in tablenames.to_series()][:-1])
tableshort = sorted([x.replace("tbl_", "") for x in tablenames])
print(tableshort)
codes_tab = pl.read_database("SELECT * from tbl_codes2names", con)
codes_tab = codes_tab.drop("recode")
all_tabs = {}
errors = {}
for tabname, shortname in zip(tablenames, tableshort):
    print("tabname:", tabname, "/// shortname:", shortname)
    tab_to_process = pl.read_database(
        f"SELECT * from {tabname}", con, infer_schema_length=None
    )
    all_tabs[tabname], errors[tabname] = codes_to_names(
        tab_to_process, codes_tab, shortname
    )

In [ ]:
for tabname, tab in all_tabs.items():
    tab = pd.DataFrame(tab, columns=tab.columns)
    exec(f"{tabname} = eval('tab')")


# DB Import


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import re
import os
con = sqlite3.connect(os.path.join(save_location, "db_mepram_sepsis_v3.sqlite3"))

cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

with open(os.path.join(save_location, "tbl_personid2center.sql"), "r") as f:
    sql_script = f.read()
    cursor.executescript(sql_script)

with open(os.path.join(save_location, "tbl_microorganismos.sql"), "r") as f:
    sql_script = f.read()
    cursor.executescript(sql_script)

tables = [row[0] for row in cursor.fetchall()]

print(tables)

dataframes = {}
for table in tables:
    print(f"Cargando la tabla: {table}")
    dataframes[table] = pd.read_sql_query(f"SELECT * FROM {table}", con)
con.close()



In [ ]:
import re
for name, table in dataframes.items():
    print(name.replace("tbl", "formulario"))
    print("Columnas: ", re.sub(r"[\[\]]", "", str([x for x  in table.columns])))


In [ ]:
for x,y in dataframes["tbl_infecciones_previas"].groupby("person_id"):
    if len(y) > 4:
        print("paciente: ", x)
        df_to_show = y
        break
print(df_to_show)

#### Function to replace outliers by NaN using IQR and recode quantitative variables into ranges from 0 to 3.  

In [ ]:
def process_variables(df, variables):
    """
    This function replaces outliers with NaN using IQR and recodes quantitative variables into ranges 0-3.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    variables (list): List of column names to process.

    Returns:
    pd.DataFrame: The DataFrame with outliers replaced by NaN and new recoded columns.
    """

    df_processed = df.copy()
    
    for var in variables:
        if var not in df.columns:
            print(f"Variable '{var}' not found in the DataFrame. It will be skipped.")
            continue

        Q1 = df_processed[var].quantile(0.25)
        Q3 = df_processed[var].quantile(0.75)
        IQR = Q3 - Q1

        lower_limit = Q1 - 2 * IQR
        upper_limit = Q3 + 2 * IQR

        outliers_mask = (df_processed[var] < lower_limit) | (df_processed[var] > upper_limit)
        outliers_count = outliers_mask.sum()
        print(f"{outliers_count} outliers replaced with NaN in the variable '{var}'.")

        df_processed.loc[outliers_mask, var] = np.nan

        new_column = f"{var}_recoded"

        df_processed[new_column] = pd.qcut(df_processed[var], 
                                           q=4, 
                                           labels=[0, 1, 2, 3], 
                                           duplicates='drop')

        df_processed[new_column] = df_processed[new_column].astype('Int64') 
        
        print(f"Variable '{var}' recoded in the column '{new_column}'.")
    
    return df_processed

# tbl_paciente

In [ ]:
df_pacientes = dataframes["tbl_paciente"].merge(dataframes["tbl_personid2center"], how="left")

df_pacientes["inf_previa_sino"] = np.where(np.isin(df_pacientes["person_id"].values, dataframes['tbl_infecciones_previas']["person_id"].values) == True, 1, 0)

bmr_previa = dataframes['tbl_infecciones_previas'].groupby("person_id")["bmr_infec_previa"].apply(lambda x: (x > 0).any())

df_pacientes = df_pacientes.merge(bmr_previa, on="person_id", how="left").fillna(False)
df_pacientes["bmr_infec_previa"] = np.where(df_pacientes["bmr_infec_previa"], 1, 0)


# tbl_colonizaciones_previas

No finalizado

# tbl_comorbilidad

In [ ]:
tbl_comorbilidad = dataframes['tbl_comorbilidad']
tbl_comorbilidad = pd.get_dummies(tbl_comorbilidad, columns=["tipo_cancer","tipo_hepatopatia"])
print(tbl_comorbilidad)

# tbl_factores_riesgo_bmr

In [ ]:
tbl_factores_riesgo_bmr = dataframes['tbl_factores_riesgo_bmr']
print(tbl_factores_riesgo_bmr)

# tbl_sintomas

1. **One-Hot Encoding con Duración**:
   - Se crea una tabla pivote donde cada síntoma es una columna, y el valor corresponde a la suma de los días que el síntoma estuvo presente durante el ingreso.
   - Los valores nulos se reemplazan por 0, indicando ausencia del síntoma.
   - Finalmente los sintomas se recategorizan en: 
     - 0: Ausencia 
     - 1: Agudo (1-7 días)
     - 2: Larga duración (>7 días)

2. **Variables de Presencia/Ausencia**:
   - A partir de la tabla de duración, se generan variables binarias que indican si el síntoma estuvo presente (1) o no (0).


In [ ]:
tbl_sintomas = dataframes['tbl_sintomas']
tbl_sintomas["sintoma"] = tbl_sintomas["sintoma"].astype(str)
tbl_sintomas["sintoma"] = "sintoma_" + tbl_sintomas["sintoma"]
tbl_sintomas_pivoted = tbl_sintomas.pivot_table(
    index=["person_id", "fecha_ingreso_urgencias"],  
    columns="sintoma",
    values="duracion_sintoma",
    aggfunc="sum",
    fill_value=0).reset_index()

presence_absence= tbl_sintomas_pivoted.copy()
presence_absence.iloc[:, 2:] = (presence_absence.iloc[:, 2:] > 0).astype(int)

recategorized= tbl_sintomas_pivoted.copy()
recategorized.iloc[:, 2:] = recategorized.iloc[:, 2:].applymap(
    lambda x:0 if x== 0 else (1 if x <= 7 else 2)
)

tbl_sintomas_complete = presence_absence.merge(
    recategorized, on= ['person_id', 'fecha_ingreso_urgencias'], suffixes= ("", "_categorico")
)

print(tbl_sintomas_complete)

# tbl_signos

In [ ]:
tbl_signos = dataframes['tbl_signos']

for col in tbl_signos.columns[2:]:
    tbl_signos[col] = tbl_signos[col].apply(lambda x: re.findall(r'\d+\.\d+|\d+', str(x)))
    tbl_signos[col] = tbl_signos[col].apply(lambda x: float(x[0]) if x else None)
    
tbl_signos['hipotermia_hipertermia'] = np.where(tbl_signos['temperatura'] >= 38, 2,
                                np.where(tbl_signos['temperatura'] >= 36, 0, 1))
tbl_signos['hipotermia_hipertermia'] = tbl_signos['hipotermia_hipertermia'].where(tbl_signos['temperatura'].notna())

#tbl_signos = tbl_signos.drop(columns=['hipotermia_hipertermia'])
tbl_signos['hipotension'] = np.where(tbl_signos['tension_arterial'].isna(), tbl_signos['hipotension'],
                                     np.where(tbl_signos['tension_arterial'] <= 100, 1, 0))
tbl_signos['taquipnea'] = np.where(tbl_signos['frec_respiratoria'].isna(), tbl_signos['taquipnea'],
                                   np.where(tbl_signos['frec_respiratoria'] > 20, 1, 0))
tbl_signos['taquicardia'] = np.where(tbl_signos['frec_cardiaca'].isna(), tbl_signos['taquicardia'],
                                     np.where(tbl_signos['frec_cardiaca'] >90, 1, 0))
tbl_signos['hipoxemia'] = np.where(tbl_signos['saturacion_o2'].isna(), tbl_signos['hipoxemia'],
                                     np.where(tbl_signos['saturacion_o2'] >90, 0, 1))
print(tbl_signos)

Remove outliers and recode

In [ ]:
vairables= ['temperatura', 'frec_respiratoria', 'frec_cardiaca', 'tension_arterial','saturacion_o2']
tbl_signos = process_variables(tbl_signos, vairables)

# tbl_sepsis

In [ ]:
tbl_sepsis = dataframes['tbl_sepsis']
tbl_sepsis['lactato_serico'] = np.where(tbl_sepsis['lactato_serico'] == '<= 2 millimole per liter', 0, 1)
for col in tbl_sepsis.columns[2:]:
    tbl_sepsis[col] = tbl_sepsis[col].apply(lambda x: re.findall(r'\d+\.\d+|\d+', str(x)))
    tbl_sepsis[col] = tbl_sepsis[col].apply(lambda x: float(x[0]) if len(x) > 0 else None)
print(tbl_sepsis)

# Remove outliers and recode

variables =['proteina_c_reactiva']
tbl_sepsis = procesar_variables(tbl_sepsis, variables)

# tbl_infecciones_previas

### Recodificación de microorganismos y dummy vars: 
Categorias restantes: "ecoli", "virus", "fungi", "Staphylococcus aureus", "Pseudomonas aeruginosa", "Klebsiella pneumoniae", "Streptococcus pneumoniae", "Enterococcus", "Enterobacteria", "other bacteria"

### Calculo de eventos y tiempo medio entre eventos/paciente

Se calcula el número de eventos por paciente (número de infecciones previas)
Se calcula el tiempo entre eventos y en el caso de pacientes con >2 eventos, se calcula el tiempo medio. 

In [ ]:
# Load table with classification group for each organism in column label
organism_classification = dataframes["tbl_microorganismos"].copy()

# Rename default values for final classification, use _ to separate target variables
label_map = {
    "NOEB": "_Other bacteria",
    "VIRUS": "_Virus",
    "FUNGUS": "_Fungi",
    "OEB": "_Enterobacteria",
    "ECOLI": "Escherichia coli",
    "SA": "Staphylococcus aureus",
    "PSA": "Pseudomonas aeruginosa",
    "KP": "Klebsiella pneumoniae",
    "SP": "Streptococcus pneumoniae",
    "EC": "Enterococcus"
}
organism_classification["label"] = organism_classification["label"].map(label_map)
# organism_classification[organism_classification.duplicated(subset="snomed_code", keep=False)].sort_values(by=["snomed_code", "label"])
# There are snomed_codes with 2 labels, so always keep the most informative
organism_classification = organism_classification.sort_values(by=["snomed_code", "label"]).drop_duplicates(subset="snomed_code", keep="first")
# Create a dictionary with {organism-snomed-code (value) : classification group (label)}
organism_codes_map = dict(zip(organism_classification["snomed_code"], organism_classification["label"]))
print(organism_codes_map)

In [ ]:
tbl_infecciones_previas_mer = dataframes['tbl_infecciones_previas'].copy()


# Pasamos a dummy los microorganismos y rellenamos la información con el sumatorio de cada dummy. De este modo no perdemos información.
# Finalmente lo almacenamos en tbl_infecciones_previas_microorganismo
tbl_infecciones_previas_mer['grupo_microorganismo'] = tbl_infecciones_previas_mer['microorganism_infec_prev'].astype(str).map(organism_codes_map)
tbl_infecciones_previas_mer = tbl_infecciones_previas_mer.drop(columns=['microorganism_infec_prev'])
tbl_infecciones_previas_mer["dummy"] = 1
tbl_infecciones_previas_mer = pd.get_dummies(tbl_infecciones_previas_mer, columns=['feno_resist_infec_prev'])
tbl_infecciones_previas_microorganismos = tbl_infecciones_previas_mer.pivot_table(
    index=["person_id", "fecha_ingreso_urgencias"],  
    columns="grupo_microorganismo",
    values="dummy",
    aggfunc="sum",
    fill_value=0).reset_index()

In [ ]:
tbl_infecciones_previas_microorganismos

In [ ]:
# Calculo del número de eventos por paciente y el tiempo medio entre cada visita. 

tbl_visitas_ip = tbl_infecciones_previas_mer.copy()
fechas_invalidas = tbl_visitas_ip[~tbl_visitas_ip['fecha_infeccion'].str.match(r'\d{4}-\d{2}-\d{2}')]
tbl_visitas_ip.loc[tbl_visitas_ip['fecha_infeccion'] == '323-05-01', 'fecha_infeccion'] = '2023-05-01'
tbl_visitas_ip['fecha_infeccion'] = pd.to_datetime(tbl_visitas_ip['fecha_infeccion'])
tbl_visitas_ip = tbl_visitas_ip[['person_id', 'fecha_ingreso_urgencias', 'fecha_infeccion']]
num_visitas = tbl_visitas_ip.groupby('person_id')['fecha_infeccion'].nunique().reset_index(name='numero_visitas')
print(num_visitas)

### Calcular tiempo de infeccion (dias) al ingreso. Tiempo medio en el caso de reinfecciones

In [ ]:
# Calculo del tiempo entre las visitas en aquellos pacientes que tenga más de 1 visita

tbl_visitas_ip = tbl_visitas_ip.sort_values(by=['person_id','fecha_infeccion'])
num_dias = tbl_visitas_ip.copy()
num_dias['numero_dias'] = num_dias.groupby('person_id')['fecha_infeccion'].diff().dt.days
num_dias = num_dias[(num_dias['numero_dias'] > 0) & (num_dias['numero_dias'].notna())]
num_dias = num_dias.groupby('person_id')['numero_dias'].mean().reset_index()

In [ ]:
# Calculo de dias desde desde la última infección hasta el ingreso

tbl_visitas_ip['fecha_ingreso_urgencias'] = pd.to_datetime(tbl_visitas_ip['fecha_ingreso_urgencias'])
ultima_infeccion = tbl_visitas_ip.groupby('person_id')['fecha_infeccion'].max().reset_index(name= 'ultima_fecha')
ultima_infeccion['ultima_fecha'] = pd.to_datetime(ultima_infeccion['ultima_fecha'])
fecha_ingreso_unica = tbl_visitas_ip[['person_id', 'fecha_ingreso_urgencias']].drop_duplicates()
ultima_infeccion_df = ultima_infeccion.merge(fecha_ingreso_unica, on='person_id')
ultima_infeccion_df['tiempo_ultima'] = (ultima_infeccion_df['fecha_ingreso_urgencias'] - ultima_infeccion_df['ultima_fecha']).dt.days

In [ ]:
# merge infecciones previas

tbl_infecciones_complete = tbl_infecciones_previas_microorganismos.merge(num_visitas, on= ['person_id'], how= 'left')
tbl_infecciones_complete = tbl_infecciones_complete.merge(num_dias, on= ['person_id'], how= 'left')
tbl_infecciones_complete = tbl_infecciones_complete.merge(ultima_infeccion_df, on= ['person_id'], how= 'left')
tbl_infecciones_complete = tbl_infecciones_complete.drop(columns=['fecha_ingreso_urgencias_y'])
print(tbl_infecciones_complete)

# Merge 

Combinación del dataset hasta este punto.

In [ ]:
df_merged = df_pacientes.merge(tbl_comorbilidad, on = ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_factores_riesgo_bmr, on = ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_sepsis, on= ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_signos, on= ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_sintomas_pivoted, on= ['person_id', 'fecha_ingreso_urgencias'], how= 'left')
df_merged = df_merged.merge(tbl_infecciones_complete, on= ['person_id'], how= 'left')

# Specifically process certain columns

In [ ]:
df_merged["numero_visitas"] = df_merged["numero_visitas"].fillna(0)
df_merged["mujer_gestante"] = df_merged["mujer_gestante"].replace("False", 0.0).astype(int)
# Process columns related with qsofa
qsofa_components = ["taquipnea", "estado_mental_alterado", "hipotension"]
df_merged["qsofa"] = df_merged["qsofa"].where(df_merged[qsofa_components].notna().all(axis=1), np.nan)

In [ ]:
df_merged

# Create html EDA report

In [ ]:
profile = ProfileReport(df_merged, title="MePRAM EDA report")
profile.to_notebook_iframe()
profile.to_file(os.path.join(save_location, "sintomas.html"))

# Plot Missing Values Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def insert_linebreak(string, lengLabel=10):
    return '\n'.join(string[i:i+lengLabel] for i in range(0, len(string), lengLabel))
merged_df = pd.read_csv(os.path.join(save_location, "df_merged.csv"))
prev_num = 0

def plot_missing_columns(subset_df, title=f"Missing Data Matrix for columns 0 to 171"):
    missing_df = subset_df.isna()

    missing_df.columns = subset_df.columns
    missing_percent = missing_df.mean() * 100
    columns_with_percent = [
        f"$\\bf{{{col}}}$ ({missing_percent[col]:.2f}%)" if missing_percent[col] > 30 else f"{col} ({missing_percent[col]:.2f}%)"
        for col in subset_df.columns
    ]
    plt.figure(figsize=(24, 6))
    ax = sns.heatmap(missing_df, vmin=0, vmax=1, cbar=False,
                xticklabels=columns_with_percent)
    ax.tick_params(axis='x', which='minor', length=40)
    plt.title(title)
    plt.xlabel("Columns (% Missing)")
    plt.ylabel("Rows", rotation=90)
    plt.xticks(rotation=90)
    plt.show()


plot_missing_columns(merged_df)

missing_df = merged_df.isna()
missing_df.columns = merged_df.columns
missing_percent = missing_df.mean() * 100
print([idx for idx,x in enumerate(missing_percent) if x > 30])
dangerous_df = merged_df.iloc[:, [idx for idx,x in enumerate(missing_percent) if x > 30]]
print(dangerous_df)
plot_missing_columns(dangerous_df, f"Missing Data Matrix of {len(dangerous_df.columns)} columns with > 30% NAs")

# Codigo para dividir por grupos de columnas
"""for new_num in range(20,len(merged_df.columns) + 20, 20):
    if new_num > len(merged_df.columns):
        new_num = len(merged_df.columns)
    subset_df = merged_df.iloc[:, prev_num:new_num]
    plot_missing_columns(subset_df)
    if new_num == len(merged_df.columns):
        break
    prev_num = new_num"""

# Create first version of sintomn-signs-sepsis tab merge

In [ ]:
print(tbl_sepsis[["person_id", "fecha_ingreso_urgencias", "sepsis", "shock_septico"]])

In [ ]:
sintom_sign_sepsis = tbl_signos.merge(tbl_sintomas_pivoted,  on=["person_id", "fecha_ingreso_urgencias"], how="left").merge(tbl_sepsis, on=["person_id", "fecha_ingreso_urgencias"], how="left")
sintom_sign_sepsis = sintom_sign_sepsis.drop("sintoma_nan", axis=1) #.fillna(value=0.0)
#sintom_sign_sepsis["sintoma"] = sintom_sign_sepsis["sintoma"].str.replace("sintoma_", "", regex=False).astype("float64")
print(sintom_sign_sepsis.columns)

In [ ]:
sintom_sign_sepsis.to_csv(os.path.join(save_location, "df_sintom_signos_v1.csv"))

# Create first database version with all params except previous events

In [ ]:
urgency_df = tbl_signos.merge(tbl_sintomas_pivoted,  on=["person_id", "fecha_ingreso_urgencias"], how="left")
urgency_df = urgency_df.merge(df_pacientes, on=["person_id", "fecha_ingreso_urgencias"], how="left")
urgency_df = urgency_df.merge(tbl_comorbilidad, on=["person_id", "fecha_ingreso_urgencias"], how="left")
urgency_df = urgency_df.merge(tbl_sepsis, on=["person_id", "fecha_ingreso_urgencias"], how="left")

In [ ]:
urgency_df.to_csv(os.path.join(save_location, "df_sin_antecedentes_v1.csv"))

In [ ]:
tot = urgency_df.shape[0]
for col in urgency_df.columns:
    na_per = 1-len(urgency_df[col].dropna())/tot
    if na_per > 0.15:
        print(f"Column {col} --> %NaN = {na_per}. Removed")

# PLOT CORRELATION GRID (PAIRGRID)

In [ ]:
sintomas_columns = [col for col in sintom_sign_sepsis.columns if "sintoma" in col]
binary_columns = [col for col in sintom_sign_sepsis.columns if sintom_sign_sepsis[col].dropna().value_counts().index.isin([0, 1]).all() and col not in sintomas_columns]
numeric_columns = [col for col in sintom_sign_sepsis.columns if sintom_sign_sepsis[col].dtype in ['int64', 'float64'] and col not in binary_columns and col not in sintomas_columns]
print(binary_columns)
print(numeric_columns)
print(sintomas_columns)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

pg = sns.PairGrid(sintom_sign_sepsis[binary_columns], hue="sepsis", palette="Set2")
pg.map_diag(sns.histplot)
pg.map_offdiag(sns.barplot)
pg.add_legend()
plt.show()

In [ ]:
sintomas_columns_group1 = sintomas_columns[0:int(len(sintomas_columns)/2)]
sintomas_columns_group1 = list(set(sintomas_columns_group1 + ["sepsis"]))
sns.color_palette("tab10")
pg = sns.PairGrid(sintom_sign_sepsis[sintomas_columns_group1], hue="sepsis", palette="Set2")
pg.map_diag(sns.histplot)
pg.map_offdiag(sns.scatterplot, alpha=0.6)
pg.add_legend()
plt.show()

In [ ]:
sintomas_columns_group2 = sintomas_columns[int(len(sintomas_columns)/2):-1]
sintomas_columns_group2 = list(set(sintomas_columns_group2 + ["sepsis"]))
pg = sns.PairGrid(sintom_sign_sepsis[sintomas_columns_group2], hue="sepsis", palette="Set2")
pg.map_diag(sns.histplot, alpha=0.5)
pg.map_offdiag(sns.scatterplot, alpha=0.6)
pg.add_legend()
plt.show()

In [ ]:
numeric_columns = list(set(numeric_columns + ["sepsis"]))
pg = sns.PairGrid(sintom_sign_sepsis[numeric_columns].drop("person_id", axis=1), hue="sepsis", palette="Set2")
pg.map_diag(sns.kdeplot, alpha=0.8)
pg.map_offdiag(sns.scatterplot, alpha=0.6)
pg.add_legend()
sns.color_palette("tab10")
plt.show()